In [ ]:
print("Hello World")

In [ ]:
import pandas as pd
import pyodbc

In [ ]:
# Connect to DataBank
connection_string = (
    "DSN=DataBank;"
    "Trusted_Connection=yes;"
)

con = pyodbc.connect(connection_string)

In [ ]:
# Write sql query
qry = "SELECT * FROM ukri_ssd_snapshot.2024_10_04_ssd_award;"

# Read sql query
df= pd.read_sql(qry, con)

In [ ]:
df['award_reference'].isna().sum()

In [ ]:
df['award_reference'] = df['award_reference'].replace('Unknown', None)


In [ ]:
df[df['award_reference'].isna()]

print(df['award_reference'].isna().sum()/len(df)*100)

# Only represents 0.13% of awards

df[df['award_reference'].isna()]['funded_amount'].sum()/df['funded_amount'].sum()*100

# or 0.15% of total funded amount so we can neglect

In [ ]:
df_dropna = df[~df['award_reference'].isna()]

In [ ]:
# award_reference is not Null and is duplicated
df_dropna[df_dropna['award_reference'].duplicated(keep=False)].sort_values(by='award_reference')['funded_amount'].sum()/df['funded_amount'].sum()*100

In [ ]:
df_dropna[df_dropna['award_reference'].duplicated(keep=False)].sort_values(by='award_reference').to_clipboard(index=False)

In [ ]:
# Those duplicated represent 0.32% of total funded amount so we neglect

df_dropna.drop_duplicates(subset='award_reference', keep=False, inplace=True)

In [ ]:
df_dropna[df_dropna['start_date'].isna() & df_dropna['end_date'].isna()]['funded_amount'].sum() / df['funded_amount'].sum()*100

In [ ]:
# Awards where there are no start or end date (so we cannot tell if they are active or not) represent 1.35% of the total funded value

In [ ]:
df_dropna = df_dropna[~(df_dropna['start_date'].isna() & df_dropna['end_date'].isna())]


In [ ]:
df_dropna[df_dropna['start_date'].isna()]

In [ ]:
df_dropna[df_dropna['end_date'].isna()]['funded_amount'].sum() # Those with end date is NaN are 0 funded amount so can neglect

In [ ]:
df_dropna = df_dropna[~df_dropna['end_date'].isna()]

In [ ]:
df_dropna[df_dropna['start_date'] > df_dropna['end_date']]['funded_amount'].sum() / df['funded_amount'].sum()*100 # negligable

In [ ]:
df_dropna = df_dropna[df_dropna['start_date'] < df_dropna['end_date']]


In [ ]:
df_dropna['funded_amount'].sum() / df['funded_amount'].sum()*100

In [ ]:
df_application = pd.read_sql("SELECT * FROM ukri_ssd_snapshot.2024_10_04_ssd_application;", con)

In [ ]:
df_application['application_category'].unique()
df_application.columns

df_application['opportunity_name'].unique()

In [ ]:
df_application[['opportunity_id', 'opportunity_name']].drop_duplicates()

In [ ]:

data_application = df_application[['application_reference', 'application_source', 'application_title', 'decision_owner', 
                                   'application_category', 'lead_organisation_id', 'organisation_source']]

In [ ]:
print(data_application[['application_reference', 'application_source']].duplicated().sum())

print(df_dropna[['application_reference', 'application_source']].duplicated().sum())
# application_reference and application_source are duplicated

In [ ]:
df_application['award_reference'] = df_application['award_reference'].replace('Unknown', None) # replace Unknown with NaN

In [ ]:

num = df_dropna[df_dropna[['application_reference', 'application_source']].duplicated(keep=False)].sort_values(by='application_reference')['funded_amount'].sum()

num / df['funded_amount'].sum()*100 # negligable

In [ ]:
df_dropna.drop_duplicates(subset=['application_reference', 'application_source'], keep=False, inplace=True)

In [ ]:
data_merge = pd.merge(df_dropna, data_application.drop_duplicates(subset=['application_reference', 'application_source'], keep=False), 
                      on=['application_reference', 'application_source'], how='left', indicator=True)

In [ ]:
data_merge[data_merge['_merge'] == 'left_only']['funded_amount'].sum() / df['funded_amount'].sum()*100 # negligable

In [ ]:
print(len(data_merge))
print(len(df_dropna))

In [ ]:
def list_financial_years(start_date, end_date):
    # Convert the input dates to pandas datetime
    start_date = pd.to_datetime(start_date, dayfirst=True)
    end_date = pd.to_datetime(end_date, dayfirst=True)
    
    # Generate a range of dates between start_date and end_date
    date_range = pd.date_range(start=start_date, end=end_date, freq='D')
    
    # Extract the financial years from the date range
    financial_years = date_range.to_series().apply(lambda x: x.year if x.month < 4 else x.year + 1).unique()
    
    return financial_years

# Example usage
start_date = '01/02/2024'
end_date = '02/07/2024'

financial_years = list_financial_years(start_date, end_date)
print("Financial years active between the dates:", financial_years) # where 2024 means FY 23/24 and 2025 means financial year 24/25

In [ ]:
def list_years(row):
    return list(range(row['start_year'], row['end_year'] + 1))

In [ ]:
data_merge['start_year'] = data_merge['start_date'].dt.to_period('Q-MAR').dt.qyear
data_merge['end_year'] = data_merge['end_date'].dt.to_period('Q-MAR').dt.qyear

# where 2024-08-01 is FY 2025 for example
# and 2026-01-31 is FY 2026

In [ ]:
data_merge['years_active'] = data_merge.apply(list_years, axis=1)


In [ ]:
data_merge['funder'].unique()

In [ ]:
data_merge['application_category'].unique()